In [19]:
import openai
from dotenv import dotenv_values
import logging
import uuid
from datetime import datetime

# Reading the .env file and storing the values in a variables
config = dotenv_values(".env")

# # Setting up the Open AI properties.
#GPT 3
openai.api_key = "4e21add22e3e4718a72cadd8d78a0336"
openai.api_type = "azure"
openai.api_base = "https://dbs-tmo-dev.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
deployment_name = "DBS-TMO-DEV"
model = "gpt-35-turbo-16k"

#GPT 4
# openai.api_key = "3f5dd45a468d4304aeebb8db0dcd672e"
# openai.api_type = "azure"
# openai.api_base ="https://dbs-tmo-poc-gpt4.openai.azure.com/"
# openai.api_version =  "2024-02-15-preview"
# deployment_name = "DBS-TMO-POC-GPT4-32K"
# model =  "gpt-4-32k"

# Model tuning parameters
temperature=1
top_p=1




In [20]:
org_requirements = str("""
RDC- High Level Overview
1. Overview
This document describes the high-level solution architecture Promotion drop feature for early EIP pay off. 
1.1. Background
EIP Payment or pay off will drop the RDC. 
•	Current State: When EIP is paid off or partially paid then RDC continues to award to full term.
•	Future State: EIP must remain open to continue receiving RDC, in line with current VZ and ATT promotional terms
1.2. Scope
The following enhancements will be done :
•	Front Ends will be enhanced to provide a warning message/pop-up during the cart phase, in the flows that may potentially close EIP (EIP is full paid) to provide information to the Expert/Rep to inform the customer of the RDC may be lost. 
•	BriteBill will be enhanced to reflect the updates of RDC change in the Bill as well as the Bill presentation at Front Ends.
•	EE will configure related rules for the updated promos impacted by the RDC change. EE will be enhanced to handle promo drop for partial/full payment on loans. The engine will also be enhanced to drop the RDC in the relevant scenario and update persisted information and provide downstream details (BAU).
•	Samson will enhance the Billing extract with additional attributes/information needed to reflect the RDC drop on the Bill presentation by Brite Bill.
•	DND will consume new customer notification events from EE and propagate the notifications to customers. BNE would create new service grades as/if needed.
1.3. Guardrails
This section provides a summary of key behaviors, rules, constrains within which the solutions will operate:
•	Systems-of-record:   
o	LPA will continue life cycle of the Promos as BAU. 
o	EE will be responsible to drop/reenroll the promo after eip payment or payment reversals.
o	DFS will be hosting logic to indicate if the promo will be lost based on the configurable date and active promotion for the loan. Loan creation date will be used to derive the promo loss indicator.
o	LPA will not automatically resume the promo if the promo is dropped by EE.
1.4. Terminology
•	DF/DFS: Device Financing/Device Financing Services specifies our device financing domain - including both our abstraction and Oracle
•	Balance payment/Unscheduled payment: A customer initiated payment made directly towards the loan balance. Balance payment is applied first within OFSLL to the open charge and then to the open balance. The result of the balance payment may be forwarded to Billing as a charge adjustment.
•	Charge Adjustment: An adjustment record generated by OFSLL and forwarded to Billing to either (a) adjust a previously forwarded charge or (b) reallocate financial transaction within Billing.
•	Promotion Credit: Credit records forwarded to billing to reduce device monthly installments. These credits are send back to OFSLL so that credit can be allocated to open charge in OFSLL
•	RDC : Recurring Device Credit.
2. High Level Solution Architecture
This section provides a birds eye view of the solution architecture. It participating domains and flows within the solution space.
NOTE: The Loan origination date after which Promo will be lost is maintained in the vault. This will be used by all the 3 apps. 
•	Flow-1: Promo drop Indicator: DFS will host a logic to provide a promo loss indicator in the agreement history and finance agreements API. The indicator will be derived based on the loan origination date and active promotions. This date needs to be configured. The T&C's for promo will be updated when the loan is originated hence loan origination date will be used to determine if the promo will be lost when any partial or full unscheduled payment is made. 
o	Agreement history 
1.	
1.	
1.	New Flag  earlyPaymentPromotionDrop in response will be sent only when the  includePromotionInfo is true in request of agreement history API.
2.	If includePromotionInfo is true then check for Loan creation date 
	If Loan creation date after T&C updated and device Promotion is active 
	set earlyPaymentPromotionDrop as true
	If Loan creation date after T&C updated and device Promotion is inactive 
	set earlyPaymentPromotionDrop as false
	If Loan creation date after T&C updated and no device promotion 
	set earlyPaymentPromotionDrop as false
	If Loan creation date is before the T&C updated 
	set earlyPaymentPromotionDrop as false
3.	If includePromotionInfo is false then no earlyPaymentPromotionDrop  in response
•	
o	Finance Agreements 
•	
o	
	
	Loan creation date after T&C updated and device Promotion is active 
	set earlyPaymentPromotionDrop is true
	Loan creation date after T&C updated and device Promotion is inactive 
	set earlyPaymentPromotionDrop is false
	Loan creation date after T&C updated and no device promotion 
	set earlyPaymentPromotionDrop is false
	Loan creation date is before the T&C updated 
	No earlyPaymentPromotionDrop indicator in response
•	Flow-2: Update Billing on Promo Drop: When promo is dropped due to early/partial payment, then new transaction type will be sent to SAMSON. This transaction will be used to identify the promo drop and can be leveraged to display the message on customers bill. TransactionReason(EE to provide the string value)will be leveraged to identify the drop reason. If its "Early payoff of a loan" or "Early partial loan payment" then create a new transaction in credit injection file to SAMSON.  
o	TransactionType: 0052
o	ActivityType: This will be actual PROMO code like TST3, TST1, BOGO etc…
o	Promo changes 
	New drop event "AwardCanceled " to be consumed by Promo billing consumer dfs-event-loan-promotion-billing. If the drop reason is "Early payoff of a loan" or "Early partial loan payment" then new record to be sent to CIG file with the new transaction type. This will be sent when the promo is dropped. 
•	Flow-3: Publish DEEP event for last monthly Charge: As per the requirement, when the last charge is injected promo should be dropped. Currently no events are published when last charge is injected but event is published after the last monthly charge payment. New DEEP event will be published when RMQ event ="loan.billing.LastMonthChargeApplied". DEEP event name =DeviceFinanceLoanLastMonthChargeApplied 
•	Flow-4: New transaction Reason code for Rev app: When the promotion is dropped due to drop reason is "Early payoff of a loan"  then the transaction reason code to Rev app would be "FULL_PAYMENT" or "Early partial loan payment" then the transaction reason code to Rev App would be "PARTIAL_PAYMENT". When the Payment is reversed with the enroll reason “Payment Reversal”, then transaction reason code to Rev app would be "REVERSE_PAYMENT"
•	Flow-5: New FAT description when the Promo is dropped : When the promotion is dropped due to drop reason is "Early payoff of a loan" then the FAT description "Pay Off Early-P" or  "Early partial loan payment" then the FAT description "Balance Payment New-P".  When the Payment is reversed, EE will publish the reason as "Payment Reversal". If the Promo is reinstated then the FAT description is  "Balance Payment Reversal - P".
•	Flow-6: Indicator of promo loss for partial payments : When the customer makes a partial payment on loan then set "earlyPaymentPromotionLoss" as true in the DEEP event LegacyLoanPayment if 
o	Loan is created after the cut off date
o	Promotion is associated with the loan and is active
o	Else 
	"earlyPaymentPromotionLoss" as false
          If the loan was created before the configured date then do not set the indicator in the DEEP payload. Please see the mapping update for the event.
If awardprofile call fails then default the earlyPaymentPromotionLoss to false.
""")



In [21]:
# analysis and goal identification for the input feature details.
temperature=0.65
top_p=1

goal_system_prompt = str("""
please act as functional analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and identify the goal.                             
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

goal_user_prompt = str("""
 Please identify the goal in the given requirements.
""")

messages = [
    {"role": "system", "content": goal_system_prompt},    
    {"role": "user", "content": goal_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
#result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
goal_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 148
prompt tokens- 1828
Total tokens- 1976
Response: The goal in the given requirements is to enhance the system to allow for the drop of the Recurring Device Credit (RDC) when Early Installment Payment (EIP) is paid off or partially paid. This enhancement aligns with the current promotional terms of other telecom providers such as Verizon (VZ) and AT&T. The system should provide warning messages/pop-ups to inform the customer and representatives about the potential loss of RDC during the cart phase. Additionally, the billing system and related applications should be updated to reflect the RDC changes and handle promo drops for partial or full payment on loans. The system should also generate notifications for customers and update the necessary records and attributes to accurately reflect the RDC drop on the bill presentation.


In [22]:
# Scope
temperature=0.65
top_p=1

scope_system_prompt = str("""
please act as business analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and identify the scope.
                            
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

scope_user_prompt = str("""
Please identify the scope in the given requirements.
""")

messages = [
    {"role": "system", "content": goal_system_prompt},    
    {"role": "user", "content": goal_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 99
prompt tokens- 1828
Total tokens- 1927
Response: The goal in the given requirements is to enhance the system to ensure that the Recurring Device Credit (RDC) is dropped when an Early Installment Payment (EIP) is paid off or partially paid off. This aligns with the current promotional terms of other telecom providers such as Verizon (VZ) and AT&T. The solution involves making enhancements to various front-end systems, billing systems, and event propagation systems to handle the promo drop, update billing information, and notify customers accordingly.


In [23]:
# Context
temperature=0.65
top_p=1

context_system_prompt = str("""
please act as functional analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and identify the context.                            
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

context_user_prompt = str("""
Please identify the context in the given requirement.
""")

messages = [
    {"role": "system", "content": context_system_prompt},    
    {"role": "user", "content": context_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 817
prompt tokens- 1828
Total tokens- 2645
Response: The context of the given requirement is the implementation of a high-level solution architecture for the Promotion drop feature for early EIP pay off in the T-Mobile system. The current state is that when EIP (Equipment Installment Plan) is paid off or partially paid, RDC (Recurring Device Credit) continues to be awarded for the full term. The future state aims to align with the current promotional terms of VZ (Verizon) and ATT (AT&T), where EIP must remain open to continue receiving RDC.

The scope of the enhancement includes:

1. Enhancements to the front ends to display a warning message/pop-up during the cart phase if the EIP may potentially be closed due to full payment, to inform the customer of potential RDC loss.
2. Enhancements to BriteBill (billing system) to reflect the RDC change in the bill and bill presentation at the front ends.
3. Configuration of rules in EE (unknown system) for u

In [24]:
# User Persona
temperature=0.65
top_p=1

persona_system_prompt = str("""
please act as functional analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and list the stakeholders who should be considered on the base of jira feature.                           
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

persona_user_prompt = str("""
Please list out the stakeholders who should be considered.
""")

messages = [
    {"role": "system", "content": persona_system_prompt},    
    {"role": "user", "content": persona_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 623
prompt tokens- 1840
Total tokens- 2463
Response: Based on the details provided in the Jira feature, the stakeholders who should be considered for this system change are:

1. Customer Architects: The customer architects who provided the high-level design are important stakeholders as they have provided the initial requirements and design specifications.

2. Front End Developers: The front end developers will be responsible for enhancing the front ends to provide warning messages/pop-ups during the cart phase, informing the customer about potential loss of RDC.

3. BriteBill Developers: The developers working on BriteBill will be responsible for enhancing the billing system to reflect the updates of RDC change in the bill and the bill presentation at front ends.

4. EE (Engine Enhancer) Developers: The developers working on the EE system will be responsible for configuring the rules for the updated promos impacted by the RDC change. They will also

In [25]:
# Rules and Conditions
temperature=0.65
top_p=1

RC_system_prompt = str("""
please act as functional analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and list out the business rules or conditions.                           
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

RC_user_prompt = str("""
Please list out the business rules or conditions.
""")

messages = [
    {"role": "system", "content": RC_system_prompt},    
    {"role": "user", "content": RC_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")





---- Tokens------
completion tokens- 771
prompt tokens- 1832
Total tokens- 2603
Response: Based on the provided high-level design details, the following business rules or conditions can be identified:

1. When EIP (Equipment Installment Plan) is paid off or partially paid, the RDC (Recurring Device Credit) should be dropped.

2. In the current state, when EIP is paid off or partially paid, RDC continues to be awarded for the full term. In the future state, EIP must remain open to continue receiving RDC, in line with current VZ and ATT promotional terms.

3. Front Ends need to be enhanced to provide a warning message/pop-up during the cart phase in flows that may potentially close EIP, to inform the customer that RDC may be lost.

4. BriteBill needs to be enhanced to reflect the updates of RDC change in the bill as well as the bill presentation at Front Ends.

5. EE (Engine Enhancements) will configure related rules for the updated promos impacted by the RDC change. It will handle promo

In [26]:
# Summerize functional requirement
temperature=0.65
top_p=1

fr_system_prompt = str("""
please act as functional analyst working for T-Mobile  which is a large Telecom provider based out of North America and you are provided with the high-level designs for a system change that needs to happen.
This high-level design is provided by the customer architects in form of a Jira feature.
Your task is to do an analysis of the design details provided and summerize the functional requirements.                           
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

fr_user_prompt = str("""
Please summerize the functional requirements.
""")

messages = [
    {"role": "system", "content": fr_system_prompt},    
    {"role": "user", "content": fr_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")





---- Tokens------
completion tokens- 731
prompt tokens- 1828
Total tokens- 2559
Response: Based on the provided Jira feature details, the functional requirements for the Promotion Drop feature for early EIP pay off can be summarized as follows:

1. Front Ends Enhancements:
   - Provide a warning message/pop-up during the cart phase to inform the Expert/Rep about the potential loss of Recurring Device Credit (RDC) when EIP is fully paid.
   
2. BriteBill Enhancements:
   - Reflect the updates of RDC change in the Bill.
   - Update the Bill presentation at Front Ends to include the RDC drop information.
   
3. EE Enhancements:
   - Configure rules for updated promos impacted by the RDC change.
   - Handle promo drop for partial/full payment on loans.
   - Enhance the engine to drop the RDC in the relevant scenario and update persisted information.
   - Provide downstream details for BAU processes.
   
4. Samson Enhancements:
   - Enhance the Billing extract with additional attributes/inf